In [2]:
import numpy as np
import pandas as pd
from scipy import io as sio

import warnings

warnings.filterwarnings('ignore')

### Load the Data

In [3]:
data = sio.loadmat('Project_data.mat')
channels = data['Channels']
test_data = data['TestData']
train_data = data['TrainData']
train_labels = data['TrainLabels'].ravel()
fs = data['fs']

In [9]:
train_labels.shape, train_data.shape, test_data.shape

((550,), (59, 5000, 550), (59, 5000, 159))

### Functions to Extract Features 

In [4]:
from sklearn.linear_model import LinearRegression
from scipy.signal import welch
from scipy.integrate import simps

def calc_variance(data):
    return np.std(data, axis=1).T

def calc_amp_hist(data, num_bins):
    num_channels = data.shape[0]
    num_trials = data.shape[2]
    features = np.zeros((num_trials, num_channels*(num_bins+1)))
    for i in range(num_trials):
        for j in range(num_channels):
            features[i, j*(num_bins+1):(j+1)*(num_bins+1)] = np.histogram(data[j, :, i], num_bins)[1]

    return features

def calc_ar(data, model_size, fit_intercept):
    num_channels = data.shape[0]
    num_trials = data.shape[2]
    signal_length = data.shape[1]

    features = np.zeros((num_trials, num_channels*(model_size + (lambda x: 1 if x else 0)(fit_intercept))))
    for i in range(num_trials):
        for j in range(num_channels):
            signal = data[j, :, i]

            feature_mat = np.zeros((signal_length-model_size, model_size))
            for k in range(model_size, signal_length):
                feature_mat[k-model_size, :] = signal[k-model_size:k]

            lr = LinearRegression(fit_intercept=fit_intercept)
            lr.fit(feature_mat, signal[model_size:])

            if fit_intercept:
                features[i, j*(model_size+1):(j+1)*(model_size+1)] = np.insert(lr.coef_, 0, lr.intercept_)
            else:
                features[i, j*(model_size):(j+1)*(model_size)] = lr.coef_

    return features

def calc_correlation(data):
    num_channels = data.shape[0]
    num_trials = data.shape[2]

    features = np.zeros((num_trials, num_channels**2))
    for i in range(num_trials):
        for j in range(num_channels):
            for k in range(num_channels):
                mean1 = np.mean(data[j, :, i])
                mean2 = np.mean(data[k, :, i])

                features[i, j*num_channels + k] = np.mean((data[j, :, i] - mean1)*(data[k, :, i])-mean2)

    return features

def calc_max_freq(data, fs): 
    num_channels = data.shape[0]
    num_trials = data.shape[2]

    features = np.zeros((num_trials, num_channels))
    for i in range(num_trials):
        for j in range(num_channels):      
            frequencies, psd = welch(data[j, :, i], fs=fs, nperseg=2048)
            frequencies = frequencies.ravel()
            features[i, j] = frequencies[np.argmax(psd)]

    return features

def calc_mean_freq(data, fs): 
    num_channels = data.shape[0]
    num_trials = data.shape[2]

    features = np.zeros((num_trials, num_channels))
    for i in range(num_trials):
        for j in range(num_channels):      
            frequencies, psd = welch(data[j, :, i], fs=fs, nperseg=2048)
            frequencies = frequencies.ravel()
            features[i, j] = np.sum(frequencies * psd) / np.sum(psd)

    return features

def calc_median_freq(data, fs): 
    num_channels = data.shape[0]
    num_trials = data.shape[2]

    features = np.zeros((num_trials, num_channels))
    for i in range(num_trials):
        for j in range(num_channels):      
            frequencies, psd = welch(data[j, :, i], fs=fs, nperseg=2048)
            frequencies = frequencies.ravel()
            cumulative_psd = np.cumsum(psd)
            median_index = np.where(cumulative_psd >= cumulative_psd[-1] / 2)[0][0]

            features[i, j] = frequencies[median_index]

    return features

def calc_rel_energy(data, fs, bands):
    num_channels = data.shape[0]
    num_trials = data.shape[2]
    
    features = np.zeros((num_trials, num_channels*len(bands.keys())))
    for i in range(num_trials):
        for j in range(num_channels):
            frequencies, psd = welch(data[j, :, i], fs=fs, nperseg=2048)
            frequencies = frequencies.ravel()

            total_energy = simps(psd, frequencies)
            
            for k, (band, (low, high)) in enumerate(bands.items()):
                idx_band = np.logical_and(frequencies >= low, frequencies <= high)
                band_energy = simps(psd[idx_band], frequencies[idx_band], axis=0)
                relative_band_energy = band_energy / total_energy

                features[i, j*len(bands.keys()) + k] = relative_band_energy

    return features

### Feature Extraction

In [79]:
ar_no_intercept = calc_ar(train_data, 10, False) #Takes lots of time to run!! Load the pre-calculated data instead!
np.save('ar_no_intercept', ar_no_intercept)

In [87]:
cross_corr = calc_correlation(train_data) #Takes lots of time to run!! Load the pre-calculated data instead!
np.save('cross_corr', cross_corr)

#### Time Features

In [8]:
var = calc_variance(train_data)
amp_hist = calc_amp_hist(train_data, 10)
ar_model = np.load('ar_no_intercept.npy')
cross_corr = np.load('cross_corr.npy')

In [15]:
var_test = calc_variance(test_data)
amp_hist_test = calc_amp_hist(test_data, 10)
ar_model_test = calc_ar(test_data, 10, False)
cross_corr_test = calc_correlation(test_data)

In [12]:
var.shape, amp_hist.shape, ar_model.shape, cross_corr.shape

((550, 59), (550, 649), (550, 590), (550, 3481))

#### Frequency Features

In [11]:
bands = {
    'Delta': (0.1, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Low-Range Beta': (12, 16),
    'Mid-Range Beta': (16, 21),
    'High-Range Beta': (21, 30),
    'Gamma': (30, 500)
}

max_freq = calc_max_freq(train_data, fs)
mean_freq = calc_mean_freq(train_data, fs)
med_freq = calc_median_freq(train_data, fs)
rel_energy = calc_rel_energy(train_data, fs, bands)

In [16]:
max_freq_test = calc_max_freq(test_data, fs)
mean_freq_test = calc_mean_freq(test_data, fs)
med_freq_test = calc_median_freq(test_data, fs)
rel_energy_test = calc_rel_energy(test_data, fs, bands)

In [13]:
max_freq.shape, mean_freq.shape, med_freq.shape, rel_energy.shape

((550, 59), (550, 59), (550, 59), (550, 413))

#### Aggregate and Save

In [18]:
np.save('var', var)
np.save('amp_hist', amp_hist)
np.save('ar_model', ar_model)
np.save('cross_corr', cross_corr)
np.save('max_freq', max_freq)
np.save('mean_freq', mean_freq)
np.save('med_freq', med_freq)
np.save('rel_energy', rel_energy)

In [19]:
np.save('var_test', var_test)
np.save('amp_hist_test', amp_hist_test)
np.save('ar_model_test', ar_model_test)
np.save('cross_corr_test', cross_corr_test)
np.save('max_freq_test', max_freq_test)
np.save('mean_freq_test', mean_freq_test)
np.save('med_freq_test', med_freq_test)
np.save('rel_energy_test', rel_energy_test)

##### JUST LOAD EVERY THING!

In [4]:
var = np.load('var.npy')
amp_hist = np.load('amp_hist.npy')
ar_model = np.load('ar_model.npy')
cross_corr = np.load('cross_corr.npy')
max_freq = np.load('max_freq.npy')
mean_freq = np.load('mean_freq.npy')
med_freq = np.load('med_freq.npy')
rel_energy = np.load('rel_energy.npy')

var_test = np.load('var_test.npy')
amp_hist_test = np.load('amp_hist_test.npy')
ar_model_test = np.load('ar_model_test.npy')
cross_corr_test = np.load('cross_corr_test.npy')
max_freq_test = np.load('max_freq_test.npy')
mean_freq_test = np.load('mean_freq_test.npy')
med_freq_test = np.load('med_freq_test.npy')
rel_energy_test = np.load('rel_energy_test.npy')

In [5]:
from sklearn.preprocessing import StandardScaler

features_tr = np.concatenate((var, amp_hist, ar_model, cross_corr, max_freq, mean_freq, med_freq, rel_energy), axis = 1)
features_te = np.concatenate((var_test, amp_hist_test, ar_model_test, cross_corr_test, max_freq_test, mean_freq_test, med_freq_test, rel_energy_test), axis = 1)

sc = StandardScaler()

features_tr = sc.fit_transform(features_tr)
features_te = sc.fit_transform(features_te)

np.save('features_train_scaled', features_tr)
np.save('features_test_scaled', features_te)

features_tr.shape, features_te.shape

((550, 5369), (159, 5369))

In [6]:
var_fea = ['variance_' + str(i) for i in range(var.shape[1])]
hist_fea = ['hist_' + str(i) for i in range(amp_hist.shape[1])]
ar_fea = ['ar_' + str(i) for i in range(ar_model.shape[1])]
cross_fea = ['correlation_' + str(i) for i in range(cross_corr.shape[1])]
max_fea = ['max_freq_' + str(i) for i in range(max_freq.shape[1])]
mean_fea = ['mean_freq_' + str(i) for i in range(mean_freq.shape[1])]
med_fea = ['median_freq_' + str(i) for i in range(med_freq.shape[1])]
energ_fea = ['rel_energy_' + str(i) for i in range(rel_energy.shape[1])]

all_fea = var_fea + hist_fea + ar_fea + cross_fea + max_fea + mean_fea + med_fea + energ_fea

def get_feature_name(index):
    return all_fea[index]

len(all_fea), get_feature_name(900)

(5369, 'ar_192')

In [7]:
features_tr.shape, train_labels.shape

((550, 5369), (550,))

### Feature Selection

In [8]:
from sklearn.feature_selection import SelectKBest

def fisher_score(X, y):
    mean = np.mean(X, axis=0)

    sb1 = (np.sum(X[y==1], axis=0) - mean)**2
    sb2 = (np.sum(X[y==-1], axis=0) - mean)**2

    sw1 = np.var(X[y==1], axis=0)
    sw2 = np.var(X[y==-1], axis=0)
    
    return sb1 + sb2 / sw1 + sw2

selector = SelectKBest(score_func=fisher_score, k=50)
X_tr = selector.fit_transform(features_tr, train_labels)

# Get the indices of the selected features
selected_indices = np.where(selector.get_support())[0]
print(f'Selected feature indices: {selected_indices}')
print(f'Selected features: {[get_feature_name(index) for index in selected_indices]}')

X_te = features_te[:, selected_indices]

Selected feature indices: [ 152  273  329  892  931  932 1002 1021 1022 1031 1101 1121 1152 1262
 1448 1521 1684 1818 2054 2241 2502 2734 3124 3188 3192 3300 3772 4015
 4138 4140 4142 4576 4584 4632 4735 4818 4825 4832 4860 4919 4943 4950
 4952 5111 5116 5234 5300 5314 5349 5363]
Selected features: ['hist_93', 'hist_214', 'hist_270', 'ar_184', 'ar_223', 'ar_224', 'ar_294', 'ar_313', 'ar_314', 'ar_323', 'ar_393', 'ar_413', 'ar_444', 'ar_554', 'correlation_150', 'correlation_223', 'correlation_386', 'correlation_520', 'correlation_756', 'correlation_943', 'correlation_1204', 'correlation_1436', 'correlation_1826', 'correlation_1890', 'correlation_1894', 'correlation_2002', 'correlation_2474', 'correlation_2717', 'correlation_2840', 'correlation_2842', 'correlation_2844', 'correlation_3278', 'correlation_3286', 'correlation_3334', 'correlation_3437', 'max_freq_39', 'max_freq_46', 'max_freq_53', 'mean_freq_22', 'median_freq_22', 'median_freq_46', 'median_freq_53', 'median_freq_55', 'rel_en

In [9]:
X_tr.shape, X_te.shape

((550, 50), (159, 50))

### Prepare the Model


In [28]:
import torch
from torch import nn, optim
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MLPNet(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size):
        super().__init__()
        self.l1 = nn.Linear(input_size, hidden1_size) 
        self.l2 = nn.Linear(hidden1_size, hidden2_size)  
        self.l3 = nn.Linear(hidden2_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.relu(self.l1(x))
        out = self.relu(self.l2(out))
        out = self.sigmoid(self.l3(out))
        return out

#### Hyper Parameters 

In [31]:
batch_size = 10
learning_rate = 0.01
input_size = 50
hidden1_size = 10
hidden2_size = 10
num_epochs = 10
num_total_steps = 550 // batch_size

model = MLPNet(input_size, hidden1_size, hidden2_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

### Train

In [32]:
X_tr_tensor = torch.tensor(np.float32(X_tr))
y_tr_tensor = torch.tensor(np.float32(np.where(train_labels == 1, 1.0, 0.0)))

kfold = KFold(n_splits=5, shuffle=True)

train_accuracies = []
valid_accuracies = []
for fold, (train_ids, valid_ids) in enumerate(kfold.split(X_tr_tensor)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)
    train_loader = DataLoader(TensorDataset(X_tr_tensor, y_tr_tensor), batch_size=batch_size, sampler=train_subsampler)
    valid_loader = DataLoader(TensorDataset(X_tr_tensor, y_tr_tensor), batch_size=batch_size, sampler=valid_subsampler)

    for epoch in range(num_epochs): 
            for i, (inputs, targets) in enumerate(train_loader):
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                outputs = model(inputs)
                
                loss = criterion(outputs.ravel(), targets)
                
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                if (i+1) % num_total_steps-1 == 0:
                    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{num_total_steps}], Loss: {loss.item():.4f}')

    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            
            predicted = torch.where(outputs > 0.5, 1, 0).T.to(device)
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    train_accuracies.append(100 * correct / total)
    print(f'Train Accuracy: {100 * correct / total:.2f}%')
    print('--------------------------------')

    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            
            predicted = torch.where(outputs > 0.5, 1, 0).T.to(device)
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    valid_accuracies.append(100 * correct / total)
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')
    print('--------------------------------')

print('\nOveral Results')
print(f'Average Training Accurcy is : {np.mean(train_accuracies)}')
print(f'Average Validation Accurcy is : {np.mean(valid_accuracies)}')

FOLD 0
--------------------------------
Epoch [1/10], Step [1/55], Loss: 0.7627
Epoch [2/10], Step [1/55], Loss: 0.5265
Epoch [3/10], Step [1/55], Loss: 0.5833
Epoch [4/10], Step [1/55], Loss: 0.4459
Epoch [5/10], Step [1/55], Loss: 0.3059
Epoch [6/10], Step [1/55], Loss: 0.6078
Epoch [7/10], Step [1/55], Loss: 0.3925
Epoch [8/10], Step [1/55], Loss: 0.3953
Epoch [9/10], Step [1/55], Loss: 0.2519
Epoch [10/10], Step [1/55], Loss: 0.3415
Train Accuracy: 86.14%
--------------------------------
Validation Accuracy: 70.91%
--------------------------------
FOLD 1
--------------------------------
Epoch [1/10], Step [1/55], Loss: 0.4066
Epoch [2/10], Step [1/55], Loss: 0.2848
Epoch [3/10], Step [1/55], Loss: 0.1387
Epoch [4/10], Step [1/55], Loss: 0.2430
Epoch [5/10], Step [1/55], Loss: 0.1774
Epoch [6/10], Step [1/55], Loss: 0.1967
Epoch [7/10], Step [1/55], Loss: 0.1757
Epoch [8/10], Step [1/55], Loss: 0.2686
Epoch [9/10], Step [1/55], Loss: 0.2818
Epoch [10/10], Step [1/55], Loss: 0.1097
T

#### Predict Labels 

In [38]:
test_data_tensor = torch.tensor(np.float32(X_te)).to(device)

model.eval()

with torch.no_grad():
    outputs = model(test_data_tensor)
        
predicted = torch.where(outputs > 0.5, 1, 0).T
np.save('test_labels_MLP', predicted.cpu())
predicted

tensor([[0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
         0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
         1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
         1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
         1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
         1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
         1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0]], device='cuda:0')

### Prepare RBF

In [65]:
import torch
from torch import nn, optim
from sklearn.model_selection import KFold
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from sklearn.cluster import KMeans

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RBF(nn.Module):
    def __init__(self, input_dim, output_dim, X_train):
        super().__init__()
        kmeans = KMeans(n_clusters=output_dim)
        kmeans.fit(X_train)
        self.centers = nn.Parameter(torch.tensor(kmeans.cluster_centers_).float())
        self.beta = nn.Parameter(torch.ones(1, output_dim))

    def radial_function(self, x):
        x = x.unsqueeze(-1)  # add an extra dimension at the end for broadcasting
        return torch.exp(-self.beta.mul((x - self.centers.T).pow(2).sum(1)))

    def forward(self, x):
        return self.radial_function(x)

class RBFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, X_train):
        super().__init__()
        self.rbf = RBF(input_dim, hidden_dim, X_train)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.rbf(x)
        return self.linear(x)

#### Hyperparameters

In [66]:
batch_size = 10
learning_rate = 0.01
input_dim = 50 
hidden_dim = 2
output_dim = 1
num_epochs = 15
num_total_steps = 550 // batch_size

model = RBFNet(input_dim, hidden_dim, output_dim, torch.tensor(np.float32(X_tr))).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

### Train

In [70]:
X_tr_tensor = torch.tensor(np.float32(X_tr))
y_tr_tensor = torch.tensor(np.float32(np.where(train_labels == 1, 1.0, 0.0)))

kfold = KFold(n_splits=5, shuffle=True)

train_accuracies = []
valid_accuracies = []
for fold, (train_ids, valid_ids) in enumerate(kfold.split(X_tr_tensor)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)
    train_loader = DataLoader(TensorDataset(X_tr_tensor, y_tr_tensor), batch_size=batch_size, sampler=train_subsampler)
    valid_loader = DataLoader(TensorDataset(X_tr_tensor, y_tr_tensor), batch_size=batch_size, sampler=valid_subsampler)
    
    for epoch in range(num_epochs): 
            for i, (inputs, targets) in enumerate(train_loader):
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                outputs = model(inputs)
                
                loss = criterion(outputs.ravel(), targets)
                
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                if (i+1) % num_total_steps-1 == 0:
                    print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{num_total_steps}], Loss: {loss.item():.4f}')

    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            
            predicted = torch.where(outputs > 0.5, 1, 0).T.to(device)
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    train_accuracies.append(100 * correct / total)
    print(f'Train Accuracy: {100 * correct / total:.2f}%')
    print('--------------------------------')

    correct, total = 0, 0
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            
            predicted = torch.where(outputs > 0.5, 1, 0).T.to(device)
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    valid_accuracies.append(100 * correct / total)
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')
    print('--------------------------------')

print('\nOveral Results')
print(f'Average Training Accurcy is : {np.mean(train_accuracies)}')
print(f'Average Validation Accurcy is : {np.mean(valid_accuracies)}')

FOLD 0
--------------------------------
Epoch [1/15], Step [1/55], Loss: 0.2322
Epoch [2/15], Step [1/55], Loss: 0.2491
Epoch [3/15], Step [1/55], Loss: 0.2432
Epoch [4/15], Step [1/55], Loss: 0.2584
Epoch [5/15], Step [1/55], Loss: 0.2615
Epoch [6/15], Step [1/55], Loss: 0.2480
Epoch [7/15], Step [1/55], Loss: 0.2376
Epoch [8/15], Step [1/55], Loss: 0.2290
Epoch [9/15], Step [1/55], Loss: 0.2280
Epoch [10/15], Step [1/55], Loss: 0.2441
Epoch [11/15], Step [1/55], Loss: 0.2375
Epoch [12/15], Step [1/55], Loss: 0.2381
Epoch [13/15], Step [1/55], Loss: 0.2649
Epoch [14/15], Step [1/55], Loss: 0.2413
Epoch [15/15], Step [1/55], Loss: 0.2489
Train Accuracy: 52.50%
--------------------------------
Validation Accuracy: 54.55%
--------------------------------
FOLD 1
--------------------------------
Epoch [1/15], Step [1/55], Loss: 0.2405
Epoch [2/15], Step [1/55], Loss: 0.2128
Epoch [3/15], Step [1/55], Loss: 0.2286
Epoch [4/15], Step [1/55], Loss: 0.2404
Epoch [5/15], Step [1/55], Loss: 0.19

#### Predict Test Labels

In [69]:
test_data_tensor = torch.tensor(np.float32(X_te)).to(device)

model.eval()

with torch.no_grad():
    outputs = model(test_data_tensor)
        
predicted = torch.where(outputs > 0.5, 1, 0).T
np.save('test_labels_RBF', predicted.cpu())
predicted

tensor([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
         1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
         0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]], device='cuda:0')